In [1]:
import pandas as pd
import numpy as np
from recommend import *

In [30]:
jester = pd.read_csv("data/jester-data-1.csv", header=None).drop(0, axis=1).T.reset_index(drop=True).T
jester_stacked = jester.stack().replace(99, np.nan).dropna().sample(200).reset_index().rename(columns={"level_0": "user", "level_1": "item", 0: "obs"})
jester_stacked

,user,item,obs
0,7035,46,5.53
1,19495,41,3.01
2,22388,68,8.16
3,12906,30,8.54
4,17074,49,6.89
...,...,...,...
195,15961,39,-3.98
196,9666,66,-7.33
197,23718,73,3.01
198,22955,35,0.49


In [29]:
sim = similarity_matrix(jester_sample)

NameError: name 'jester_sample' is not defined

In [2]:
def decision(rating):
    if rating >= 5:
        return True
    else:
        return False

In [4]:
def eval_random(data, size = 200, func = mean_utility, n = None):
    stacked = data.stack().replace(99, np.nan).dropna().sample(size).reset_index().rename(columns={"level_0": "user", "level_1": "item", 0: "obs_rating"})
    sample = data.loc[stacked["user"].unique(), stacked["item"].unique()].sort_index().T.sort_index().T.replace(99, np.nan)

    sim = similarity_matrix(sample)

    results = stacked.copy()
    results['pred_rating'] = pd.Series(stacked.index).apply(lambda i: func(sample, sim, stacked.loc[i, "user"], stacked.loc[i, "item"],
                                                                    method="user", n=n))

    results['delta_rating'] = np.abs(results['obs_rating'] - results['pred_rating'])
    results['obs_rec'] = results['obs_rating'].apply(decision)
    results['pred_rec'] = results['pred_rating'].apply(decision)

    return results

In [76]:
def eval_list(data, samp_list, func = mean_utility, n = None):
    stacked = samp_list.copy()
    stacked['obs_rating'] = pd.Series(stacked.index).apply(lambda i: data.loc[stacked.loc[i, 'user'], stacked.loc[i, 'item']]).replace(99, np.nan)
    stacked = stacked.dropna().reset_index(drop=True)
    sample = data.loc[stacked["user"].unique(), stacked["item"].unique()].sort_index().T.sort_index().T.replace(99, np.nan)

    sim = similarity_matrix(sample)

    results = stacked.copy()
    results['pred_rating'] = pd.Series(stacked.index).apply(lambda i: func(sample, sim, stacked.loc[i, "user"], stacked.loc[i, "item"],
                                                                    method="user", n=n))

    results['delta_rating'] = np.abs(results['obs_rating'] - results['pred_rating'])
    results['obs_rec'] = results['obs_rating'].apply(decision)
    results['pred_rec'] = results['pred_rating'].apply(decision)

    return results

In [70]:
samp_list = pd.DataFrame()
samp_list['user'] = pd.Series(list(jester.index)).sample(200, replace=True).reset_index(drop=True)
samp_list['item'] = pd.Series(list(jester.columns)).sample(200, replace=True).reset_index(drop=True)
samp_list.to_csv("samp_list.csv", index=False, header=False)

In [77]:
results = eval_list(jester, samp_list, func=adj_wtd_sum, n=5)
results

,user,item,obs_rating,pred_rating,delta_rating,obs_rec,pred_rec
0,20987,36,-9.37,0.524486,9.894486,False,False
1,3907,2,8.64,6.395765,2.244235,True,True
2,20785,64,7.09,-1.191069,8.281069,True,False
3,17223,46,-9.71,-1.492426,8.217574,False,False
4,11021,4,3.35,2.785828,0.564172,False,False
...,...,...,...,...,...,...,...
141,12618,82,7.23,4.546899,2.683101,True,False
142,17221,4,7.77,-1.693260,9.463260,True,False
143,1509,80,3.35,1.525548,1.824452,False,False
144,11808,24,-7.48,-3.716825,3.763175,False,False


In [66]:
conf_matrix = pd.crosstab(results['pred_rec'], results['obs_rec'])
conf_matrix = conf_matrix.reindex(index=conf_matrix.columns, columns=conf_matrix.columns,
                                  fill_value=0).rename_axis("obs", axis=1).rename_axis("pred", axis=0)
conf_matrix

obs,False,True
pred,,
False,110,27
True,5,7


In [55]:
jester

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,...,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,...,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00
24981,99.00,99.00,99.00,99.00,-9.71,99.00,4.56,-8.30,99.00,99.00,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00


In [49]:
results = eval_random(jester, func=adj_wtd_sum, n=5)
results

,user,item,obs_rating,pred_rating,delta_rating,obs_rec,pred_rec
0,1223,14,3.50,3.719545,0.219545,False,False
1,18254,40,-6.80,1.993253,8.793253,False,False
2,13747,54,-1.60,1.164319,2.764319,False,False
3,2728,78,8.98,5.526254,3.453746,True,True
4,4299,88,9.22,2.273586,6.946414,True,False
...,...,...,...,...,...,...,...
195,1100,7,-8.50,1.021315,9.521315,False,False
196,14043,67,1.50,0.125856,1.374144,False,False
197,14267,94,8.30,1.286119,7.013881,True,False
198,14780,4,3.83,0.980313,2.849687,False,False


In [38]:
pd.crosstab(results['obs_rec'], results['pred_rec'])

pred_rec,False
obs_rec,
False,90
True,50


In [39]:
results['delta_rating'].mean()

4.420880872530111

In [24]:
jester_sample = jester.loc[jester_stacked["user"].unique(), jester_stacked["item"].unique()].sort_index().T.sort_index().T.replace(99, np.nan)
jester_sample

,0,1,2,3,4,5,6,7,8,9,...,89,91,92,93,94,95,96,97,98,99
94,8.16,2.28,4.37,7.91,1.12,9.03,2.23,5.24,7.28,9.13,...,3.06,8.93,8.88,3.40,9.13,7.77,3.69,-2.62,1.75,1.80
164,NaN,6.36,9.27,7.04,4.85,9.27,5.24,-9.51,-1.70,-2.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,-9.71,-8.25,-8.30,-2.62,-8.93,-0.05,2.18,4.47,1.21,2.38,...,0.29,-9.22,-6.50,3.25,2.62,1.99,8.74,-6.55,-0.53,-8.59
253,9.03,2.43,3.98,-5.05,-3.40,1.12,2.52,-5.73,-3.35,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.29,NaN,0.63
290,-0.24,0.78,-7.14,-0.24,-0.58,-0.97,-7.04,-8.40,-7.04,-7.18,...,4.27,5.92,2.52,4.22,5.10,5.29,4.47,1.55,2.72,-7.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24147,-1.89,-1.41,0.05,-7.57,-4.32,3.11,3.59,3.59,2.91,3.93,...,4.08,-0.97,3.59,3.74,2.67,3.35,3.74,-0.15,3.40,2.14
24217,7.23,-0.53,-7.86,-7.67,0.92,-8.11,5.83,-8.20,-8.98,1.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24298,8.11,-0.24,-8.88,-9.32,9.27,-8.45,-7.04,5.68,2.18,1.65,...,9.32,-7.14,1.46,-6.41,0.15,1.26,-0.10,-0.24,-4.76,-4.76
24442,5.05,-0.49,-4.71,-0.44,2.96,-3.93,5.63,0.87,3.98,1.36,...,2.86,0.53,0.49,-6.80,4.90,4.81,2.14,-8.30,2.48,6.17


In [25]:
sim = similarity_matrix(jester_sample)
jester_stacked['pred'] = pd.Series(jester_stacked.index).apply(lambda i: adj_wtd_sum(jester_sample, sim,
                                                                                     jester_stacked.loc[i, "user"],
                                                                                     jester_stacked.loc[i, "item"],
                                                                                     method="user", n=5))
jester_stacked

,user,item,obs,pred
0,24217,11,-8.35,-3.344119
1,6997,42,-6.31,-2.831599
2,7294,72,3.25,1.561554
3,24147,71,0.00,2.326186
4,4233,39,-8.88,1.057527
...,...,...,...,...
195,8345,29,4.22,0.904519
196,14773,14,-0.19,0.241655
197,10545,61,6.02,0.472953
198,8054,85,0.73,5.991950
